In [11]:
import pandas as pd
import sys
from tqdm.notebook import tqdm
from PIL import Image
sys.path.append('/home/docker_current/py_files/utils')
from ruCLIP import RUCLIPPredictor

train_images_path = "/home/docker_current/datasets/train"
df_train = pd.read_csv("/home/docker_current/datasets/train.csv")
df_train['text_features'] = None
df_train['image_features'] = None
df_train['pred_id'] = 0
df_train.head(3)

,id,description,object_img,text_features,image_features,pred_id
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,None,None,0
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,None,None,0
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,None,None,0


In [12]:
predictor = RUCLIPPredictor()

In [13]:
for ind in tqdm(range(len(df_train))):
    sample_image_path = "/home/docker_current/datasets/train/" + str(df_train['object_img'][ind]) + ".png"
    image = Image.open(sample_image_path)
    text = df_train['description'][ind]

    text_features, image_features = predictor.inference(text, image)
    df_train['text_features'][ind] = text_features
    df_train['image_features'][ind] = image_features

  0%|          | 0/2098 [00:00<?, ?it/s]

1it [00:00, 57.19it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00, 61.70it/s]
1it [00:00, 50.16it/s]
1it [00:00, 45.25it/s]
1it [00:00, 49.19it/s]
1it [00:00, 15.79it/s]
1it [00:00, 21.41it/s]
1it [00:00, 17.53it/s]
1it [00:00, 17.93it/s]
1it [00:00, 15.50it/s]
1it [00:00, 27.90it/s]
1it [00:00, 16.50it/s]
1it [00:00, 15.22it/s]
1it [00:00, 18.36it/s]
1it [00:00, 16.00it/s]
1it [00:00, 40.07it/s]
1it [00:00, 46.24it/s]
1it

In [23]:
import numpy as np
text_feat_array = np.vstack((i for i in df_train['text_features']))
image_feat_array = np.vstack((i for i in df_train['image_features']))

base_path = "/home/docker_current/py_files/embeddings/"
name_emb_file = base_path + "ruclip_vit_large_patch14-336.npy"

with open(name_emb_file, 'wb') as f:
    np.save(f, text_feat_array)
    np.save(f, image_feat_array)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
image_feat_array.shape, text_feat_array.shape

((2098, 768), (2098, 768))

In [17]:
#Чекинг сохраненных эмбеддингов

# with open(name_emb_file, 'rb') as f:
#     l_text_feat_array = np.load(f)
#     l_image_feat_array = np.load(f)

In [10]:
#Чекинг косинусной близости

# import torch
# cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
# ind = 3

# cos(torch.Tensor(df_train['text_features'][ind]), torch.Tensor(df_train['image_features'][ind] ))

tensor([0.2248])